In [1]:
_ = !pip install pandas
_ = !pip install langchain=0.0.193
_=  !pip install openai

In [2]:
_= !pip install langchain --upgrade

In [3]:
import os
import sqlite3
import pandas as pd
from langchain import OpenAI, SQLDatabase, SQLDatabaseChain

In [16]:
data_path = '/content/drive/MyDrive/Colab Notebooks/Vehicle_Collisions.xlsx'

In [18]:
df = pd.read_excel(data_path, header=0)
df.head(15)

,CRASH DATE,CRASH TIME,BOROUGH,ZIP CODE,LATITUDE,LONGITUDE,LOCATION,ON STREET NAME,CROSS STREET NAME,OFF STREET NAME,...,CONTRIBUTING FACTOR VEHICLE 2,CONTRIBUTING FACTOR VEHICLE 3,CONTRIBUTING FACTOR VEHICLE 4,CONTRIBUTING FACTOR VEHICLE 5,COLLISION_ID,VEHICLE TYPE CODE 1,VEHICLE TYPE CODE 2,VEHICLE TYPE CODE 3,VEHICLE TYPE CODE 4,VEHICLE TYPE CODE 5
0,2021-11-09,0.399306,BROOKLYN,11208.0,40.667202,-73.866500,"(40.667202, -73.8665)",NaN,NaN,1211 LORING AVENUE,...,NaN,NaN,NaN,NaN,4456314,Sedan,NaN,NaN,NaN,NaN
1,2021-11-12,0.821528,BRONX,10463.0,40.872620,-73.904686,"(40.87262, -73.904686)",WEST KINGSBRIDGE ROAD,HEATH AVENUE,NaN,...,Unspecified,NaN,NaN,NaN,4487040,Station Wagon/Sport Utility Vehicle,Sedan,NaN,NaN,NaN
2,2021-11-12,0.197917,MANHATTAN,10001.0,40.748917,-73.993546,"(40.748917, -73.993546)",NaN,NaN,232 WEST 30 STREET,...,Unspecified,NaN,NaN,NaN,4486905,Station Wagon/Sport Utility Vehicle,NaN,NaN,NaN,NaN
3,2021-12-12,0.375000,QUEENS,11385.0,40.704470,-73.901480,"(40.70447, -73.90148)",NaN,NaN,59-14 67 AVENUE,...,Unspecified,NaN,NaN,NaN,4486933,Station Wagon/Sport Utility Vehicle,NaN,NaN,NaN,NaN
4,2022-12-07,0.743056,BROOKLYN,11225.0,40.663303,-73.960490,"(40.663303, -73.96049)",NaN,NaN,44 EMPIRE BOULEVARD,...,Unspecified,NaN,NaN,NaN,4545699,Sedan,NaN,NaN,NaN,NaN
5,2022-08-03,0.833333,BROOKLYN,11207.0,40.666256,-73.900215,"(40.666256, -73.900215)",NaN,NaN,360 SNEDIKER AVENUE,...,Unspecified,NaN,NaN,NaN,4522285,Sedan,NaN,NaN,NaN,NaN
6,2022-12-04,0.830556,BROOKLYN,11203.0,40.650110,-73.930214,"(40.65011, -73.930214)",UTICA AVENUE,SNYDER AVENUE,NaN,...,NaN,NaN,NaN,NaN,4522136,Station Wagon/Sport Utility Vehicle,NaN,NaN,NaN,NaN
7,2022-07-04,0.593056,MANHATTAN,10017.0,40.748158,-73.970330,"(40.748158, -73.97033)",1 AVENUE,EAST 41 STREET,NaN,...,Passing or Lane Usage Improper,NaN,NaN,NaN,4522242,Sedan,Station Wagon/Sport Utility Vehicle,NaN,NaN,NaN
8,2021-09-12,0.847222,BROOKLYN,11223.0,40.592070,-73.962990,"(40.59207, -73.96299)",EAST 7 STREET,CRAWFORD AVENUE,NaN,...,Unspecified,NaN,NaN,NaN,4485150,Bike,NaN,NaN,NaN,NaN
9,2021-09-12,0.114583,QUEENS,11422.0,40.653023,-73.738950,"(40.653023, -73.73895)",149 AVENUE,HUXLEY STREET,NaN,...,NaN,NaN,NaN,NaN,4485026,Sedan,NaN,NaN,NaN,NaN


In [20]:
df_for_analysis = df[['CRASH DATE', 'CRASH TIME','BOROUGH',
                      'ON STREET NAME','CROSS STREET NAME','OFF STREET NAME',
                      'NUMBER OF PERSONS INJURED','NUMBER OF PERSONS KILLED',
                      'NUMBER OF PEDESTRIANS KILLED','NUMBER OF PEDESTRIANS INJURED',
                      'NUMBER OF MOTORIST KILLED', 'CONTRIBUTING FACTOR VEHICLE 1', 'VEHICLE TYPE CODE 1']]
df_for_analysis.head(5)

,CRASH DATE,CRASH TIME,BOROUGH,ON STREET NAME,CROSS STREET NAME,OFF STREET NAME,NUMBER OF PERSONS INJURED,NUMBER OF PERSONS KILLED,NUMBER OF PEDESTRIANS KILLED,NUMBER OF PEDESTRIANS INJURED,NUMBER OF MOTORIST KILLED,CONTRIBUTING FACTOR VEHICLE 1,VEHICLE TYPE CODE 1
0,2021-11-09,0.399306,BROOKLYN,NaN,NaN,1211 LORING AVENUE,0,0,0,0,0,Unspecified,Sedan
1,2021-11-12,0.821528,BRONX,WEST KINGSBRIDGE ROAD,HEATH AVENUE,NaN,1,0,0,0,0,Unspecified,Station Wagon/Sport Utility Vehicle
2,2021-11-12,0.197917,MANHATTAN,NaN,NaN,232 WEST 30 STREET,0,0,0,0,0,Following Too Closely,Station Wagon/Sport Utility Vehicle
3,2021-12-12,0.375000,QUEENS,NaN,NaN,59-14 67 AVENUE,0,0,0,0,0,Passing Too Closely,Station Wagon/Sport Utility Vehicle
4,2022-12-07,0.743056,BROOKLYN,NaN,NaN,44 EMPIRE BOULEVARD,0,0,0,0,0,Oversized Vehicle,Sedan


In [31]:
conn = sqlite3.connect('Vehicle_Collisions.sqlite')
cur = conn.cursor()

table_name = 'Vehicle_Collisions'
query = f"""CREATE TABLE IF NOT EXISTS {table_name}
                                                    (CRASH_DATE Date, CRASH_TIME Time, BOROUGH Text,ON_STREET_NAME Text, CROSS_STREET_NAME Text, OFF_STREET_NAME Text, NUMBER_OF_PERSONS_INJURED integer, NUMBER_OF_PERSONS_KILLED integer, NUMBER_OF_PEDESTRIANS_KILLED integer, NUMBER_OF_PEDESTRIANS_INJURED integer, NUMBER_OF_MOTORIST_KILLED integer,CONTRIBUTING_FACTOR_VEHICLE_1 integer, VEHICLE_TYPE_CODE_1 integer)"""
cur.execute(query)
conn.commit()

df_for_analysis.to_sql(table_name, conn, if_exists='replace', index=False)
conn.commit()

# somehow the header ends up as a database record. Remove them
# cur.execute(f"""DELETE FROM {table_name} WHERE model='model';""")
conn.commit()
conn.close()

In [32]:
!ls | grep Vehicle_Collisions.sqlite

Vehicle_Collisions.sqlite


In [33]:
def read_query(sql):
    conn = sqlite3.connect('Vehicle_Collisions.sqlite')
    cur = conn.cursor()
    cur.execute(sql)
    rows = cur.fetchall()
    for row in rows:
        print(row)
    conn.close()

In [34]:
read_query('SELECT * FROM Vehicle_Collisions LIMIT 10;')

('2021-11-09 00:00:00', 0.3993055555555556, 'BROOKLYN', None, None, '1211      LORING AVENUE', 0, 0, 0, 0, 0, 'Unspecified', 'Sedan')
('2021-11-12 00:00:00', 0.8215277777777777, 'BRONX', 'WEST KINGSBRIDGE ROAD', 'HEATH AVENUE', None, 1, 0, 0, 0, 0, 'Unspecified', 'Station Wagon/Sport Utility Vehicle')
('2021-11-12 00:00:00', 0.19791666666666666, 'MANHATTAN', None, None, '232       WEST 30 STREET', 0, 0, 0, 0, 0, 'Following Too Closely', 'Station Wagon/Sport Utility Vehicle')
('2021-12-12 00:00:00', 0.375, 'QUEENS', None, None, '59-14     67 AVENUE', 0, 0, 0, 0, 0, 'Passing Too Closely', 'Station Wagon/Sport Utility Vehicle')
('2022-12-07 00:00:00', 0.7430555555555555, 'BROOKLYN', None, None, '44        EMPIRE BOULEVARD', 0, 0, 0, 0, 0, 'Oversized Vehicle', 'Sedan')
('2022-08-03 00:00:00', 0.8333333333333334, 'BROOKLYN', None, None, '360       SNEDIKER AVENUE', 0, 0, 0, 0, 0, 'Unspecified', 'Sedan')
('2022-12-04 00:00:00', 0.8305555555555556, 'BROOKLYN', 'UTICA AVENUE', 'SNYDER AVENUE',

In [35]:
db = SQLDatabase.from_uri('sqlite:///Vehicle_Collisions.sqlite')

In [36]:
os.environ['OPENAI_API_KEY'] = 'YOUR_API_KEY'
llm = OpenAI(temperature=0)

In [37]:
db_chain = SQLDatabaseChain(llm=llm, database=db, verbose=True)

/usr/local/lib/python3.10/dist-packages/langchain/chains/sql_database/base.py:63: UserWarning: Directly instantiating an SQLDatabaseChain with an llm is deprecated. Please instantiate with llm_chain argument or using the from_llm class method.
  warnings.warn(


In [40]:
db_chain.run('I want to know the count of all persons who are killed in the BOROUGHS for the year 2022?')




> Entering new  chain...
I want to know the count of all persons who are killed in the BOROUGHS for the year 2022?
SQLQuery:SELECT COUNT("NUMBER OF PERSONS KILLED") FROM "Vehicle_Collisions" WHERE "CRASH DATE" >= date('2022-01-01') AND "CRASH DATE" < date('2023-01-01');
SQLResult: [(18512,)]
Answer:18512 persons were killed in the BOROUGHS in the year 2022.
> Finished chain.


'18512 persons were killed in the BOROUGHS in the year 2022.'